In [1]:
import pandas as pd
import sqlalchemy as sa
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns

# Set some display options for Pandas
pd.set_option('display.max_columns', 50)

# Load environment variables from .env file located in the parent directory
# This allows us to connect to the database securely without hardcoding credentials
load_dotenv(dotenv_path='../.env')

def get_db_engine():
    """Creates and returns a SQLAlchemy engine."""
    try:
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")
        db_host = os.getenv("DB_HOST")
        db_port = os.getenv("DB_PORT")
        db_name = os.getenv("DB_NAME")
        
        engine = sa.create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
        print("Database engine created successfully.")
        return engine
    except Exception as e:
        print(f"Error creating database engine: {e}")
        return None

engine = get_db_engine()

# SQL query to load the data, joining tables to get location names
sql_query = """
SELECT 
    r.reading_date,
    r.aqi,
    r.pm10,
    r.pm25,
    r.o3,
    r.no2,
    r.temperature_celsius,
    r.precipitation_mm,
    r.wind_speed_kmh,
    l.city,
    l.country
FROM 
    daily_readings r
JOIN 
    locations l ON r.location_id = l.id
ORDER BY
    l.city, r.reading_date;
"""

# Load the data into a pandas DataFrame
if engine:
    try:
        df = pd.read_sql(sql_query, engine, parse_dates=['reading_date'])
        print("Data loaded successfully into DataFrame.")
        display(df.head())
    except Exception as e:
        print(f"Error loading data: {e}")

Database engine created successfully.
Data loaded successfully into DataFrame.


,reading_date,aqi,pm10,pm25,o3,no2,temperature_celsius,precipitation_mm,wind_speed_kmh,city,country
0,2025-06-08,52,61.50,17.15,25.00,3.78,28.4,0.4,21.8,Pooler,USA
1,2025-06-08,55,55.98,17.58,25.02,0.91,27.9,0.3,20.0,Port Wentworth,USA
2,2025-06-08,52,62.10,12.00,25.00,4.90,28.2,0.0,23.8,Savannah,USA
